In [ ]:
import plotly.graph_objects as go
import pycountry
import pandas as pd

df = pd.read_csv('temp_data.csv').melt(id_vars=['year'], var_name='country', value_name='temperature')
keys = [list(pycountry.countries)[i].alpha_3 for i in range(len(pycountry.countries))]
names = [list(pycountry.countries)[i].name for i in range(len(pycountry.countries))]
conv = { v:k for k,v in zip(keys,names) }
def match_name(x):
    try:
        return conv[x]
    except KeyError as e:
        for  n in names:
            if x in n:
                return conv[n]
        return 'N/A'
df['codes'] = df.apply(lambda x: match_name(x['country']), axis=1)

In [ ]:
years = df['year'].unique()
years.sort()
years_sliders = []

for year in years:
    tdf = df[df['year'] == year]

    years_sliders.append(go.Choropleth(
        visible=False,
        locations = tdf['codes'],
        z = tdf['temperature'],
        text = tdf['country'],
        colorscale = 'RdBu',
        reversescale=True,
        marker_line_color='darkgray',
        marker_line_width=0.5,
        colorbar_tickprefix = 'C',
        colorbar_title = 'Temperature',
        zmin=df['temperature'].min(), zmax=df['temperature'].max(), zmid=0
    ))

fig = go.Figure(data=years_sliders)

steps = []
for i in range(len(years)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": "Year: " + str(years[i])}],
              label=str(years[i])  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active=len(years),
    currentvalue={"prefix": "Steps: "},
    pad={"t": 50},
    steps=steps,
)]

geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='orthographic'
    )
fig.update_layout(
    sliders=sliders,
    geo=geo
)
fig.data[0].visible = True
fig.write_html('../plots/simple_globe.html')

fig.show()